In [ ]:
import numpy as np
import tensorflow.keras.optimizers
import tensorflow.keras.losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Activation, Dropout
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

***Loading the MNIST dataset from keras***

In [ ]:
import keras
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 2s 0us/step


In [ ]:
x_test.ndim

3

***We need to normalize the train and test data. The goal of normalization is to change the values of numeric columns in the dataset to a common scale, without distorting differences in the ranges of values.***

In [ ]:
X_train, X_test = x_train/255.0, x_test/255.0

***Now let's perform one-hot encoding on the train and test labels.***

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
train_encoded = y_train.reshape(-1,1)
Y_train = ohe.fit_transform(train_encoded)

test_encoded = y_test.reshape(-1,1)
Y_test = ohe.fit_transform(test_encoded)

In [ ]:
Y_train.shape

(60000, 10)

In [ ]:
model = Sequential()

model.add(CuDNNLSTM(64, input_shape = X_train.shape[1:],return_sequences=True))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(128))
model.add(Activation("relu"))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dense(10,activation='softmax'))
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 28, 64)            24064     
_________________________________________________________________
activation (Activation)      (None, 28, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 28, 64)            0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128)               99328     
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8

In [ ]:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001,decay=1e-5)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=32, epochs=5, validation_data=(X_test, Y_test))

test_loss, test_acc = model.evaluate(X_test, Y_test)
print("Test Loss: {0} - Test Acc: {1}".format(test_loss, test_acc))

Epoch 1/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0584 - accuracy: 0.9832 - val_loss: 0.0524 - val_accuracy: 0.9835
Epoch 2/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0506 - accuracy: 0.9856 - val_loss: 0.0467 - val_accuracy: 0.9864
Epoch 3/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0448 - accuracy: 0.9872 - val_loss: 0.0431 - val_accuracy: 0.9877
Epoch 4/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.0382 - accuracy: 0.9889 - val_loss: 0.0487 - val_accuracy: 0.9854
Epoch 5/5
313/313 [==============================] - 2s 8ms/step - loss: 0.0456 - accuracy: 0.9877
Test Loss: 0.04558902233839035 - Test Acc: 0.9876999855041504
